In [29]:
import cv2 as cv
import numpy as np
import os

# 取色後RGB值的字典
color_dictionary = {}

# 上色函數
def fill_color_demo(sketch, color_dictionary):
    copyIma = sketch.copy()
    h, w = sketch.shape[:2]
    mask = np.zeros([h+2, w+2], np.uint8)
    for position, rgb in color_dictionary.items():
        x, y = position
        r, g, b = rgb
        cv.floodFill(copyIma, mask, (x, y), (int(b), int(g), int(r)), (100, 100, 100), (100,100,100), cv.FLOODFILL_FIXED_RANGE)
    cv.imshow("Filled Sketch", copyIma)
    cv.waitKey(0)

def convert_to_jpg(image_path):
    # 讀取圖片
    img = cv.imread(image_path)
    # 取得圖片名稱和擴展名
    image_name, ext = os.path.splitext(image_path)
    # 如果不是JPG格式，則進行轉換
    if ext.lower() != '.jpg':
        jpg_path = image_name + '.jpg'
        cv.imwrite(jpg_path, img)
        return jpg_path
    else:
        return image_path

def face_detect(image_path, cascade_path, sketch, original_color):
    # 將圖片轉換為JPG格式
    img_path = convert_to_jpg(image_path)
    # 讀取彩色圖片
    img = cv.imread(img_path)
    # 進行臉部偵測的處理
    # 1. 轉換成灰度圖像（用於臉部偵測）
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 2. 進行直方圖均衡化
    img_gray = cv.equalizeHist(img_gray)
    # 載入臉部偵測器模型
    face_cascade = cv.CascadeClassifier(cascade_path)
    # 進行臉部偵測
    faces = face_cascade.detectMultiScale(img_gray)
    # 在原始彩色圖像上畫出臉部矩形框
    for (x, y, w, h) in faces:
        # 畫出矩形框
        img = cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 255), 5)
        # 計算矩形框的重心座標
        centroid_x = x + w // 2
        centroid_y = y + h // 2
        # 將重心座標標註在圖像上
        cv.circle(img, (centroid_x, centroid_y), 5, (0, 255, 0), -1)
        # 標註文字
        cv.putText(img, f'({centroid_x}, {centroid_y})', (centroid_x + 10, centroid_y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        # 獲取原始彩色圖像中對應位置的顏色
        b, g, r = original_color[centroid_y, centroid_x]
        # 將顏色添加到顏色字典中
        color_dictionary[(centroid_x, centroid_y)] = (r, g, b)
    
    # 在原始彩色圖像上畫出臉部矩形框和座標
    for (x, y, w, h) in faces:
        # 畫出矩形框
        original_color = cv.rectangle(original_color, (x, y), (x+w, y+h), (255, 0, 255), 5)
        # 計算矩形框的重心座標
        centroid_x = x + w // 2
        centroid_y = y + h // 2
        # 將重心座標標註在圖像上
        cv.circle(original_color, (centroid_x, centroid_y), 5, (0, 255, 0), -1)
        # 標註文字
        cv.putText(original_color, f'({centroid_x}, {centroid_y})', (centroid_x + 10, centroid_y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 調用上色函數进行填充
    fill_color_demo(sketch, color_dictionary)

    # 顯示結果
    cv.imshow('Face detection', img)
    cv.imshow('Original Color', original_color)
    cv.waitKey(0)

if __name__ == "__main__":
    # 圖片路徑
    sketch_path = "./girl_sketch.jpg" # 圖片原始格式為PNG
    original_path = "./girl_original.png"  # 原始彩色图像路径
    cascade_path = "./lbpcascade_animeface.xml"  # 臉部偵測器模型路徑
    
    # 讀取原始彩色圖像
    original_color = cv.imread(original_path)
    
    # 讀取素描圖像
    sketch = cv.imread(sketch_path)

    # 呼叫臉部偵測函式
    face_detect(sketch_path, cascade_path, sketch, original_color)

    cv.destroyAllWindows()
